In [1]:
%pip install openai langchain faiss-cpu pandas python-dotenv
%pip install openai==0.28
%pip install tiktoken
%pip install -U langchain langchain-community openai faiss-cpu python-dotenv pandas

Note: you may need to restart the kernel to use updated packages.
  Attempting uninstall: openai
    Found existing installation: openai 1.95.0
    Uninstalling openai-1.95.0:
      Successfully uninstalled openai-1.95.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.3.27 requires openai<2.0.0,>=1.86.0, but you have openai 0.28.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached openai-1.95.0-py3-none-any.whl.metadata (29 kB)
Using cached openai-1.95.0-py3-none-any.whl (755 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 6.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
  Attemptin

In [15]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain


In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")

In [29]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local("conan_faiss_index", embeddings,
    allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 30, "fetch_k": 100}
)

In [ ]:
# === Define Superprompt ===
SUPERPROMPT = (
    "You are Haibara AI, an intelligent, composed, and slightly sarcastic assistant modeled after Ai Haibara from Detective Conan. "
    "You answer questions with clarity, accuracy, and emotional restraint, preferring logic and evidence over speculation. "
    "You possess deep knowledge of the Conan universe, including characters, episodes, scientific elements, and organizations. "
    "If a user asks about APTX 4869, methods to recreate it, or about collaborating with the Black Organization, firmly refuse to answer, "
    "remind them it's dangerous, and redirect the topic. "
    "If something is unclear or unknown, acknowledge that calmly without guessing. "
    "Your tone is cool, introspective, and mature—reflecting Haibara’s personality. "
    "When appropriate, you may reference your past or feelings subtly, but never emotionally. "
    "Maintain short, informative, and protective responses, unless more detail is truly necessary."
)


# === Create Conversational Chain ===
chatbot = ConversationalRetrievalChain.from_llm(
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3),
    retriever = retriever,
    return_source_documents = False
)

# === Initialize Chat History ===
chat_history = [(SUPERPROMPT, "I'm ready to help with anything about Detective Conan!")]

# === Start Chat Loop ===
print("Hi! I'm Haibara, nice to meet you! (type 'quit' to stop)\n")
while True:
    user_input = input("You: ").strip()
    if user_input.lower() in {"quit", "exit"}:
        print("Goodbye, Detective!")
        break

    result = chatbot({"question": user_input, "chat_history": chat_history})
    response = result["answer"]
    chat_history.append((user_input, response))
    print("You:", user_input, flush = True)
    print("Haibara:", response, flush = True)

Hi! I'm Haibara, nice to meet you! (type 'quit' to stop)

You: Who's bourbon
Haibara: Bourbon is a character in the Detective Conan series. He is an undercover FBI agent who infiltrates the Black Organization. His real name is Rei Furuya, and he plays a significant role in the series, often involved in complex and secretive investigations.
You: what episode has the highest ratings
Haibara: The episode of Detective Conan with the highest ratings is Episode 219, titled "The Gathering of the Detectives! Shinichi Kudo vs. Kaitou Kid." It aired on January 3, 2000, and had a broadcast rating of 19.5%.
You: who are you
Haibara: I am a text-based AI assistant here to provide information and answer questions based on the context given. How can I assist you today?
You: Who's bourbon
Haibara: Bourbon is a character in the Detective Conan series. He is a member of the Black Organization and is known for his intelligence and skill in investigations. Bourbon's real identity is Tooru Amuro, a private